In [47]:
import pandas as pd
#from uszipcode import SearchEngine
import math
import numpy as np

In [26]:
joined = pd.read_csv("../data/joined_zip.csv")
joined.drop(joined.columns[0], axis=1, inplace=True)
joined.head()

,zips,PM 2.5,PM 10,Ozone,Nitrogen dioxide,Sulfur dioxide,Asthma,Asthma ED visits (0-18 years),Asthma ED visits (0-4 years),Asthma ED Visits (65+ years),Chronic lower respiratory disease deaths,Diagnosed heart disease,Hypertension,Lung cancer incidence,Screened or diagnosed with COPD
0,60302,47.523705,2.864987,83.976009,51.295768,417.744167,13.7,246.0,340.5,47.1,35.2,6.30,35.8,85.2,15.2
1,60304,45.479591,2.864987,58.631936,34.618608,570.996734,13.7,246.0,340.5,47.1,35.2,6.30,35.8,85.2,15.2
2,60402,42.682097,2.864987,74.805807,63.930207,477.971690,13.7,59.6,63.9,11.7,30.9,13.30,43.5,66.2,24.6
3,60415,1.104920,2.864987,138.329414,37.042564,729.686756,13.0,24.0,20.4,3.2,43.1,12.35,11.5,85.6,24.0
4,60453,1.104920,2.864987,76.440655,37.051921,736.780428,13.0,24.0,20.4,3.2,43.1,12.35,11.5,85.6,24.0


In [41]:
joined = pd.read_csv("../data/joined_zip.csv", index_col=0, usecols=list(range(1,16)))
joined.head()

,PM 2.5,PM 10,Ozone,Nitrogen dioxide,Sulfur dioxide,Asthma,Asthma ED visits (0-18 years),Asthma ED visits (0-4 years),Asthma ED Visits (65+ years),Chronic lower respiratory disease deaths,Diagnosed heart disease,Hypertension,Lung cancer incidence,Screened or diagnosed with COPD
zips,,,,,,,,,,,,,,
60302,47.523705,2.864987,83.976009,51.295768,417.744167,13.7,246.0,340.5,47.1,35.2,6.30,35.8,85.2,15.2
60304,45.479591,2.864987,58.631936,34.618608,570.996734,13.7,246.0,340.5,47.1,35.2,6.30,35.8,85.2,15.2
60402,42.682097,2.864987,74.805807,63.930207,477.971690,13.7,59.6,63.9,11.7,30.9,13.30,43.5,66.2,24.6
60415,1.104920,2.864987,138.329414,37.042564,729.686756,13.0,24.0,20.4,3.2,43.1,12.35,11.5,85.6,24.0
60453,1.104920,2.864987,76.440655,37.051921,736.780428,13.0,24.0,20.4,3.2,43.1,12.35,11.5,85.6,24.0


In [4]:
#WHO Air Quality Guidelines
#PM2.5 (ug/m3)
AQG_PM2_5 = 10
IT1_PM2_5 = 35
IT2_PM2_5 = 25
IT3_PM2_5 = 15

#PM10 (ug/m3)
AQG_PM10 = 20
IT1_PM10 = 70
IT2_PM10 = 50
IT3_PM10 = 30

#Ozone O3 (ug/m3)
AQG_O3 = 100
IT1_O3 = 160

#SO2 (ug/m3)
AQG_SO2 = 20
IT1_SO2 = 125
IT2_SO2 = 50

#NO2 (ug/m3)
AQG_NO2 = 40


In [5]:
aqg = pd.DataFrame({'AQG': ['AQG_PM2_5', 'AQG_PM10', 'AQG_O3', 'AQG_SO2', 'AQG_NO2'], 'value': [10, 20, 100, 20, 40], 'mortality': [6, 3, .5, .3, .5]})


In [7]:
aqg.to_csv('/Users/erinmckeon/cse6242/Project/aqg.csv')

In [ ]:
joined.

In [6]:
asthma = joined['Asthma'].mean()
asthmaed0 = joined['Asthma ED visits (0-18 years)'].mean()
asthmaed1 = joined['Asthma ED visits (0-4 years)'].mean()
asthmaed2 = joined['Asthma ED Visits (65+ years)'].mean()
resp = joined['Chronic lower respiratory disease deaths'].mean()
heart = joined['Diagnosed heart disease'].mean()
hyper = joined['Hypertension'].mean()
lung = joined['Lung cancer incidence'].mean()
copd = joined['Screened or diagnosed with COPD'].mean()

print(asthma)
print(asthmaed0)
print(asthmaed1)
print(asthmaed2)
print(resp)
print(heart)
print(hyper)
print(lung)
print(copd)

10.591525423728813
126.23389830508474
142.25423728813556
31.574576271186437
27.42302259898305
9.329661016949155
28.691807909491526
65.62316384186441
20.85084745762712


In [9]:
#Calculation factor weights, based on increment of increased mortality per increase in 5ug/m3
wpm25 = 6 #increase in 5ug/m3 = 6% mortality
wpm10 = 3 #increase in 5ug/m3 = 3% mortality
wo3 = .5 #increase in 5ug/m3 = 0.5% mortality
wso2 = .3 #increase in 5ug/m3 = 0.3% mortality
wno2 = .5 #increase in 5ug/m3 = 0.52% mortality


In [50]:
# joined.loc[int('60604')]['Asthma']
# np.any(joined.index.isin([60633]))
joined.index

Int64Index([60302, 60304, 60402, 60415, 60453, 60456, 60459, 60601, 60602,
            60603, 60604, 60605, 60606, 60607, 60608, 60609, 60610, 60611,
            60612, 60613, 60614, 60615, 60616, 60617, 60618, 60619, 60620,
            60621, 60622, 60623, 60624, 60625, 60628, 60629, 60630, 60632,
            60634, 60636, 60637, 60638, 60639, 60640, 60641, 60642, 60643,
            60644, 60647, 60649, 60651, 60652, 60653, 60654, 60655, 60657,
            60659, 60660, 60661, 60804, 60805],
           dtype='int64', name='zips')

In [ ]:
#Calculate Livability Score
for i in [0]:
    liv = 100    
    PM_liv = joined['PM 2.5'][i] / AQG_PM2_5
    liv -= PM_liv * wpm25
    print('PM_liv: ', PM_liv, PM_liv * wpm25)
    
    #PM10
    PM1_liv = joined['PM 10'][i] / AQG_PM10
    liv -= PM1_liv * wpm10
    print('PM1_liv: ', PM1_liv, PM_liv * wpm10)
    
    #Ozone
    O_liv = joined['Ozone'][i]  / AQG_O3
    liv -= O_liv * wo3
    print('O_liv: ', O_liv, O_liv * wo3)
            
    #Nitrogen Dioxide
    NO2_liv = joined['Nitrogen dioxide'][i] / AQG_NO2
    liv -= NO2_liv * wno2
    print('NO2_liv: ', NO2_liv, NO2_liv * wno2)

    #Sulfur Dioxide
    SO2_liv = joined['Sulfur dioxide'][i] / AQG_SO2
    liv -= SO2_liv * wso2
    print('SO2_liv: ', SO2_liv, SO2_liv * wso2)
    
    #Health Data
    #Asthma
    ast = joined['Asthma'][i] / asthma
    liv -= ast
    print('ast: ', ast)
    
    ast1 = joined['Asthma ED visits (0-18 years)'][i] / asthmaed0
    liv -= ast1
    print('ast1: ', ast1)
        
    ast2 = joined['Asthma ED visits (0-4 years)'][i] / asthmaed1
    liv -= ast2
    print('ast2: ', ast2)
        
    ast3 = joined['Asthma ED Visits (65+ years)'][i] / asthmaed2
    liv -= ast3
    print('ast3: ', ast3)
    
    #Chronic lower respiratory disease
    res = joined['Chronic lower respiratory disease deaths'][i] / resp
    liv -= res
    print('res: ', res)
    
    #Heart Disease
    hea = joined['Diagnosed heart disease'][i] / heart
    liv -= hea
    print('hea: ', hea)
    
    #Hypertension
    hyp = joined['Hypertension'][i] / hyper
    liv -= hyp
    print('hyp: ', hyp)
    
    #Lung Cancer Incidence
    lun = joined['Lung cancer incidence'][i] / lung
    liv -= lun
    print('lun: ', lun)
    
    #Screened or diagnosed with COPD
    cop = joined['Screened or diagnosed with COPD'][i] / copd
    liv -= cop
    print('cop: ', cop)
print(liv)

In [ ]:
#Calculate Livability Score
liv_score = []
for i in list(range(len(joined))):
    liv = 100    
    PM_liv = joined['PM 2.5'][i] / AQG_PM2_5
    liv -= PM_liv * wpm25
    
    #PM10
    PM1_liv = joined['PM 10'][i] / AQG_PM10
    liv -= PM1_liv * wpm10
    
    #Ozone
    O_liv = joined['Ozone'][i]  / AQG_O3
    liv -= O_liv * wo3
            
    #Nitrogen Dioxide
    NO2_liv = joined['Nitrogen dioxide'][i] / AQG_NO2
    liv -= NO2_liv * wno2

    #Sulfur Dioxide
    SO2_liv = joined['Sulfur dioxide'][i] / AQG_SO2
    liv -= SO2_liv * wso2
    
    #Health Data
    #Asthma
    ast = joined['Asthma'][i] / asthma
    liv -= ast
    
    ast1 = joined['Asthma ED visits (0-18 years)'][i] / asthmaed0
    liv -= ast1
        
    ast2 = joined['Asthma ED visits (0-4 years)'][i] / asthmaed1
    liv -= ast2
        
    ast3 = joined['Asthma ED Visits (65+ years)'][i] / asthmaed2
    liv -= ast3
    
    #Chronic lower respiratory disease
    res = joined['Chronic lower respiratory disease deaths'][i] / resp
    liv -= res
    
    #Heart Disease
    hea = joined['Diagnosed heart disease'][i] / heart
    liv -= hea
    
    #Hypertension
    hyp = joined['Hypertension'][i] / hyper
    liv -= hyp
    
    #Lung Cancer Incidence
    lun = joined['Lung cancer incidence'][i] / lung
    liv -= lun
    
    #Screened or diagnosed with COPD
    cop = joined['Screened or diagnosed with COPD'][i] / copd
    liv -= cop
    if liv > 0:
        liv_score.append(liv)
    else:
        liv_score.append(0)

In [ ]:
livpd = pd.Series(liv_score, name = "liv_score")

In [ ]:
joined = joined.join(livpd)

In [ ]:
joined.head()

In [ ]:
joined.to_csv('/Users/erinmckeon/cse6242/Project/joined_liv.csv')